# Machine Learning Pipeline - Feature Engineering

In the following notebooks, we will go through the implementation of each one of the steps in the Machine Learning Pipeline. 

We will discuss:

1. Data Analysis
2. **Feature Engineering**
3. Feature Selection
4. Model Training
5. Obtaining Predictions / Scoring

## load packages and utilities

In [1]:
# to handle datasets
import os
import pandas as pd
import numpy as np

# for plotting
import matplotlib.pyplot as plt

# for the yeo-johnson transformation
import scipy.stats as stats

# to divide train and test set
from sklearn.model_selection import train_test_split

# feature scaling
from sklearn.preprocessing import MinMaxScaler

# to save the trained scaler class
import joblib

# import functions from data analysis
from data_analysis_utils import *

# to visualise al the columns in the dataframe
pd.pandas.set_option('display.max_columns', None)

## load dataset

In [2]:
# load dataset
csvname = 'cumc+csbc_t1gd+t2+adc_predKi67ForMayo_pyradiomics.csv'
datadir = os.path.join(os.getcwd(), 'dataset')
data = pd.read_csv(os.path.join(datadir, csvname))

target = 'Ki67 LI'
indexcol = 'biopsyImage'
drop_vars = ['Source', 'AnonID']
cat_vars = ['Sex', 'Primary/Recurrent']
num_vars =  [var for var in data.columns if var not in cat_vars+drop_vars+[target, indexcol]]

if data.index.name is not indexcol:
    data.set_index(indexcol, inplace=True)
print('size baseline:', data.shape)

# 1. deal with extra vars
data = drop_select_vars(data, drop_vars)
print('size after cleaning:', data.shape)

size baseline: (318, 5250)
size after cleaning: (318, 5248)


## Separate dataset into train and test

It is important to separate our data intro training and testing set. 

When we engineer features, some techniques learn parameters from data. It is important to learn these parameters only from the train set. This is to avoid over-fitting.

**Separating the data into train and test involves randomness, therefore, we need to set the seed to ensure reproducibility between runs.**

In [12]:
# Let's separate into train and test set
# Remember to set the seed (random_state for this sklearn function)

X_train, X_test, y_train, y_test = train_test_split(
    data.drop([target], axis=1), # predictive variables
    data[target], # target
    test_size=0.1, # portion of dataset to allocate to test set
    random_state=0, # we are setting the seed here
)
y_train = pd.DataFrame(columns=[target], index=X_train.index.values, data=y_train)
y_test = pd.DataFrame(columns=[target], index=X_test.index.values, data=y_test)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((286, 5247), (32, 5247), (286, 1), (32, 1))

In [13]:
# transform target
target_map = get_target_transformation_plan(y_train, target)
y_train = apply_transform_num_var_plan(y_train, target_map)
y_test = apply_transform_num_var_plan(y_test, target_map)

# fill up missing vals
missing_plan = get_fill_missing_plan(X_train)
if missing_plan is not None:
    X_train = apply_fill_missing_plan(X_train, missing_plan)
    X_test = apply_fill_missing_plan(X_test, missing_plan)
    
# transform numeric predictors
num_mappings = get_numericvars_transformation_plan(X_train, num_vars)
X_train = apply_transform_num_var_plan(X_train, num_mappings)
X_test = apply_transform_num_var_plan(X_test, num_mappings)

# transformm categorical predictors
cat_mappings = {'Sex': {'Unknown': 0, 'F': 1, 'M': 2}, 
                'Primary/Recurrent': {'Unknown': 0, 'Primary':1, 'Recurrent': 2}}
X_train = apply_transform_cat_var_plan(X_train, cat_mappings)
X_test = apply_transform_cat_var_plan(X_test, cat_mappings)

# cat vars with na:  0
# num vars with na:  5245
                                   FillValue
firstorder_10Percentile_t1gd         2.49043
firstorder_90Percentile_t1gd         4.34773
firstorder_Energy_t1gd               855.696
firstorder_Entropy_t1gd               4.6914
firstorder_InterquartileRange_t1gd  0.851431
notskewed, modskewed, extskewed: [0, 707, 3748]
focusing on modskewed ..
--> fixable, nochange, worsened after yeojohnnson: [702, 5, 0]

focusing on no change and worsened ones (without 0)
--> fixable, no_change, worsened after log: [0, 0, 4]
--> if still not fixable, binarize
YeoJohnson 702
Binarize 4543


## Feature Scaling

For use in linear models, features need to be either scaled. We will scale features to the minimum and maximum values:

In [14]:
# create scaler
scaler = MinMaxScaler()

#  fit  the scaler to the train set
scaler.fit(X_train) 

# transform the train and test set
X_train = pd.DataFrame(
    scaler.transform(X_train),
    columns=X_train.columns,
    index=X_train.index.values
)

X_test = pd.DataFrame(
    scaler.transform(X_test),
    columns=X_train.columns,
    index=X_test.index.values
)

In [15]:
X_train.head()

,firstorder_10Percentile_t1gd,firstorder_90Percentile_t1gd,firstorder_Energy_t1gd,firstorder_Entropy_t1gd,firstorder_InterquartileRange_t1gd,firstorder_Kurtosis_t1gd,firstorder_Maximum_t1gd,firstorder_MeanAbsoluteDeviation_t1gd,firstorder_Mean_t1gd,firstorder_Median_t1gd,firstorder_Minimum_t1gd,firstorder_Range_t1gd,firstorder_RobustMeanAbsoluteDeviation_t1gd,firstorder_RootMeanSquared_t1gd,firstorder_Skewness_t1gd,firstorder_TotalEnergy_t1gd,firstorder_Uniformity_t1gd,firstorder_Variance_t1gd,glcm_Autocorrelation_t1gd,glcm_ClusterProminence_t1gd,glcm_ClusterShade_t1gd,glcm_ClusterTendency_t1gd,glcm_Contrast_t1gd,glcm_Correlation_t1gd,glcm_DifferenceAverage_t1gd,glcm_DifferenceEntropy_t1gd,glcm_DifferenceVariance_t1gd,glcm_Id_t1gd,glcm_Idm_t1gd,glcm_Idmn_t1gd,glcm_Idn_t1gd,glcm_Imc1_t1gd,glcm_Imc2_t1gd,glcm_InverseVariance_t1gd,glcm_JointAverage_t1gd,glcm_JointEnergy_t1gd,glcm_JointEntropy_t1gd,glcm_MaximumProbability_t1gd,glcm_SumAverage_t1gd,glcm_SumEntropy_t1gd,glcm_SumSquares_t1gd,glszm_GrayLevelNonUniformity_t1gd,glszm_GrayLevelNonUniformityNormalized_t1gd,glszm_GrayLevelVariance_t1gd,glszm_HighGrayLevelZoneEmphasis_t1gd,glszm_LargeAreaEmphasis_t1gd,glszm_LargeAreaHighGrayLevelEmphasis_t1gd,glszm_LargeAreaLowGrayLevelEmphasis_t1gd,glszm_LowGrayLevelZoneEmphasis_t1gd,glszm_SizeZoneNonUniformity_t1gd,glszm_SizeZoneNonUniformityNormalized_t1gd,glszm_SmallAreaEmphasis_t1gd,glszm_SmallAreaHighGrayLevelEmphasis_t1gd,glszm_SmallAreaLowGrayLevelEmphasis_t1gd,glszm_ZoneEntropy_t1gd,glszm_ZonePercentage_t1gd,glszm_ZoneVariance_t1gd,gldm_DependenceEntropy_t1gd,gldm_DependenceNonUniformity_t1gd,gldm_DependenceNonUniformityNormalized_t1gd,gldm_DependenceVariance_t1gd,gldm_GrayLevelNonUniformity_t1gd,gldm_GrayLevelVariance_t1gd,gldm_HighGrayLevelEmphasis_t1gd,gldm_LargeDependenceEmphasis_t1gd,gldm_LargeDependenceHighGrayLevelEmphasis_t1gd,gldm_LargeDependenceLowGrayLevelEmphasis_t1gd,gldm_LowGrayLevelEmphasis_t1gd,gldm_SmallDependenceEmphasis_t1gd,gldm_SmallDependenceHighGrayLevelEmphasis_t1gd,gldm_SmallDependenceLowGrayLevelEmphasis_t1gd,glrlm_GrayLevelNonUniformity_t1gd,glrlm_GrayLevelNonUniformityNormalized_t1gd,glrlm_GrayLevelVariance_t1gd,glrlm_HighGrayLevelRunEmphasis_t1gd,glrlm_LongRunEmphasis_t1gd,glrlm_LongRunHighGrayLevelEmphasis_t1gd,glrlm_LongRunLowGrayLevelEmphasis_t1gd,glrlm_LowGrayLevelRunEmphasis_t1gd,glrlm_RunEntropy_t1gd,glrlm_RunLengthNonUniformity_t1gd,glrlm_RunLengthNonUniformityNormalized_t1gd,glrlm_RunPercentage_t1gd,glrlm_RunVariance_t1gd,glrlm_ShortRunEmphasis_t1gd,glrlm_ShortRunHighGrayLevelEmphasis_t1gd,glrlm_ShortRunLowGrayLevelEmphasis_t1gd,ngtdm_Busyness_t1gd,ngtdm_Coarseness_t1gd,ngtdm_Complexity_t1gd,ngtdm_Contrast_t1gd,ngtdm_Strength_t1gd,log-sigma-5-0-mm-3D_firstorder_10Percentile_t1gd,log-sigma-5-0-mm-3D_firstorder_90Percentile_t1gd,log-sigma-5-0-mm-3D_firstorder_Energy_t1gd,log-sigma-5-0-mm-3D_firstorder_Entropy_t1gd,log-sigma-5-0-mm-3D_firstorder_InterquartileRange_t1gd,log-sigma-5-0-mm-3D_firstorder_Kurtosis_t1gd,log-sigma-5-0-mm-3D_firstorder_Maximum_t1gd,log-sigma-5-0-mm-3D_firstorder_MeanAbsoluteDeviation_t1gd,log-sigma-5-0-mm-3D_firstorder_Mean_t1gd,log-sigma-5-0-mm-3D_firstorder_Median_t1gd,log-sigma-5-0-mm-3D_firstorder_Minimum_t1gd,log-sigma-5-0-mm-3D_firstorder_Range_t1gd,log-sigma-5-0-mm-3D_firstorder_RobustMeanAbsoluteDeviation_t1gd,log-sigma-5-0-mm-3D_firstorder_RootMeanSquared_t1gd,log-sigma-5-0-mm-3D_firstorder_Skewness_t1gd,log-sigma-5-0-mm-3D_firstorder_TotalEnergy_t1gd,log-sigma-5-0-mm-3D_firstorder_Uniformity_t1gd,log-sigma-5-0-mm-3D_firstorder_Variance_t1gd,log-sigma-5-0-mm-3D_glcm_Autocorrelation_t1gd,log-sigma-5-0-mm-3D_glcm_ClusterProminence_t1gd,log-sigma-5-0-mm-3D_glcm_ClusterShade_t1gd,log-sigma-5-0-mm-3D_glcm_ClusterTendency_t1gd,log-sigma-5-0-mm-3D_glcm_Contrast_t1gd,log-sigma-5-0-mm-3D_glcm_Correlation_t1gd,log-sigma-5-0-mm-3D_glcm_DifferenceAverage_t1gd,log-sigma-5-0-mm-3D_glcm_DifferenceEntropy_t1gd,log-sigma-5-0-mm-3D_glcm_DifferenceVariance_t1gd,log-sigma-5-0-mm-3D_gl

In [16]:
# let's now save the train and test sets for the next notebook!
X_train.to_csv(os.path.join(datadir, 'xtrain.csv'), index=True)
X_test.to_csv(os.path.join(datadir, 'xtest.csv'), index=True)

y_train.to_csv(os.path.join(datadir, 'ytrain.csv'), index=True)
y_test.to_csv(os.path.join(datadir, 'ytest.csv'), index=True)

In [17]:
# now let's save the scaler
joblib.dump(scaler, 'minmax_scaler.joblib') 

['minmax_scaler.joblib']